In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

In [5]:
import json
import pandas as pd
import numpy as np
import os
import glob
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import balanced_accuracy_score
from datetime import datetime, timedelta
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
import psycopg2
from sklearn.preprocessing import StandardScaler
# from .env import db info
load_dotenv()

db_password = os.getenv("db_password")
db_username = os.getenv("db_username")
db_url = os.getenv("db_url")
db_port = os.getenv("db_port")
db_name = os.getenv("db_name")
db_table = os.getenv("db_table")

import time

In [6]:
#create the connection to the PostgreSQL database, then add the movies_df DataFrame to a SQL database.
db_string = f"postgresql://{db_username}:{db_password}@{db_url}:{db_port}/{db_name}"
engine = create_engine(db_string)
#out.to_sql(name=db_table, con=engine, if_exists='append',index=False)

In [8]:
df = pd.read_sql(f"SELECT * FROM sweetdealtrip WHERE days_before_purchase < 100;", con=engine)

In [9]:
df.head()

,time_stamp,trip_id,depart_airport,return_airport,depart_date,return_date,total_cost,depart_datetime,depart_duration,depart_stops,return_datetime,return_duration,return_stops,days_before_purchase
0,2022-06-23 04:41:02,SNA-PHX-2022-06-30-2022-07-04,SNA,PHX,2022-06-30,2022-07-04,269,2022-06-30 12:56:00,78,0,2022-07-04 10:18:00,88,0,6
1,2022-06-23 04:41:52,SNA-PHX-2022-07-07-2022-07-11,SNA,PHX,2022-07-07,2022-07-11,239,2022-07-07 13:11:00,79,0,2022-07-11 10:18:00,88,0,13
2,2022-06-23 04:42:43,SNA-PHX-2022-07-14-2022-07-18,SNA,PHX,2022-07-14,2022-07-18,300,2022-07-14 13:11:00,79,0,2022-07-18 14:00:00,79,0,20
3,2022-06-23 04:43:34,SNA-PHX-2022-07-21-2022-07-25,SNA,PHX,2022-07-21,2022-07-25,300,2022-07-21 16:44:00,84,0,2022-07-25 10:18:00,88,0,27
4,2022-06-23 04:54:55,LAX-ORD-2022-06-30-2022-07-04,LAX,ORD,2022-06-30,2022-07-04,579,2022-06-30 08:05:00,242,0,2022-07-04 15:20:00,252,0,6


In [10]:
df = df[['total_cost','depart_duration','depart_stops','return_duration','return_stops','days_before_purchase']]

In [11]:
columns = ["total_cost","depart_duration",
           "depart_stops","return_duration","return_stops",
           "days_before_purchase"
    
]

target = ["days_before_purchase"]

In [12]:
# Create our features
# X = df2.drop(columns='total_cost')
X = df[["total_cost"]] #, "depart_airport", "return_airport", "return_duration", "return_stops", "depart_duration"
# X.head()
# X = pd.get_dummies(X)
X.head()
# Create our target
y = df.loc[:, target].copy()

In [13]:
# file_path = Path("tmintest.csv")
# df2 = pd.read_csv(file_path, skiprows=1)[:-2]
# df2 = df.loc[:, columns].copy()
# df2.head()

In [14]:
# # Create our features
# X = df2.drop(columns='total_cost')
# X = pd.get_dummies(X)
# # Create our target
# y = df2.loc[:, target].copy()

In [15]:
# # Create our features
# # X = df2.drop(columns='total_cost')
# X = df2[["total_cost", "return_duration", "return_stops", "depart_duration"]] #, "depart_airport", "return_airport"
# # X.head()
# X = pd.get_dummies(X)
# X.head()
# # Create our target
# y = df2.loc[:, target].copy()

In [16]:
X.describe()

,total_cost
count,1543.000000
mean,831.775762
std,564.230543
min,178.000000
25%,472.000000
50%,677.000000
75%,918.000000
max,3297.000000


In [19]:
# Check the balance of our target values
y['days_before_purchase'].value_counts()

5     25
10    22
17    22
24    22
12    21
      ..
78     7
85     7
92     7
99     7
1      6
Name: days_before_purchase, Length: 100, dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(1157, 1)

# Ensemble Learners

# Balanced Random Forest Classifier

In [21]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
X_train_scaled.shape, y_train.shape

((1157, 1), (1157, 1))

In [23]:
# Resample the training data with the BalancedRandomForestClassifier

clf = BalancedRandomForestClassifier(n_estimators=128, random_state=78) 
# Fitting the model
clf = clf.fit(X_train_scaled, y_train)

In [24]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = clf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.01020408163265306

In [25]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [26]:
# Print the imbalanced classification report
# YOUR CODE HERE

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00         4
          1       0.00      0.00      1.00      0.00      0.00      0.00         4
          2       0.00      0.00      1.00      0.00      0.00      0.00         1
          3       0.01      1.00      0.00      0.02      0.00      0.00         4
          4       0.00      0.00      1.00      0.00      0.00      0.00         3
          5       0.00      0.00      1.00      0.00      0.00      0.00         6
          6       0.00      0.00      1.00      0.00      0.00      0.00         1
          7       0.00      0.00      1.00      0.00      0.00      0.00         3
          8       0.00      0.00      1.00      0.00      0.00      0.00         3
          9       0.00      0.00      1.00      0.00      0.00      0.00         5
         10       0.00      0.00      1.00      0.00      0.00      0.00         6
   

In [27]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
feature_names = X.columns
sorted(zip(clf.feature_importances_, feature_names), reverse=True)

[(1.0, 'total_cost')]

In [28]:
X.columns

Index(['total_cost'], dtype='object')

In [29]:
# Need to create Bins or EEC will take way too damn long

# Easy Ensemble AdaBoost Classifier

In [35]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE
eec = EasyEnsembleClassifier(n_estimators = 100,random_state=1)
eec = eec.fit(X_train, y_train)

In [36]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.002551020408163265

In [37]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[0, 2, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 3, 0, ..., 0, 0, 0],
       [0, 3, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0]], dtype=int64)

In [38]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00         4
          1       0.01      0.25      0.56      0.01      0.38      0.14         4
          2       0.00      0.00      1.00      0.00      0.00      0.00         1
          3       0.00      0.00      1.00      0.00      0.00      0.00         4
          4       0.00      0.00      1.00      0.00      0.00      0.00         3
          5       0.00      0.00      1.00      0.00      0.00      0.00         6
          6       0.00      0.00      1.00      0.00      0.00      0.00         1
          7       0.00      0.00      1.00      0.00      0.00      0.00         3
          8       0.00      0.00      1.00      0.00      0.00      0.00         3
          9       0.00      0.00      1.00      0.00      0.00      0.00         5
         10       0.00      0.00      1.00      0.00      0.00      0.00         6
   